# 데이터 전처리

In [1]:
df = pd.read_csv("noshow.csv")

In [2]:
# df.rename(columns = {"No-show": "No_show"}, inplace=True)

### Age 범주형 변수로 바꾸기

In [3]:
df["Age_bin"] = "0"
df["Age_bin"][df.Age < 10] = "0s"
df["Age_bin"][(df.Age >= 10) & (df.Age < 20)] = "10s"
df["Age_bin"][(df.Age >= 20) & (df.Age < 30)] = "20s"
df["Age_bin"][(df.Age >= 30) & (df.Age < 40)] = "30s"
df["Age_bin"][(df.Age >= 40) & (df.Age < 50)] = "40s"
df["Age_bin"][(df.Age >= 50) & (df.Age < 60)] = "50s"
df["Age_bin"][(df.Age >= 60) & (df.Age < 70)] = "60s"
df["Age_bin"][(df.Age >= 70) & (df.Age < 80)] = "70s"
df["Age_bin"][(df.Age >= 80) & (df.Age < 90)] = "80s"
df["Age_bin"][df.Age >= 90] = "90s"

### 날짜형식 범주형 변수로 바꿔주기

In [4]:
df["Scheduled_date"] = df["ScheduledDay"].apply(lambda x: x[:10])
df['Scheduled_date'] = pd.to_datetime(df['Scheduled_date'])
df['Scheduled_time'] = df['ScheduledDay'].apply(lambda x: x[11:-1])
df['Scheduled_time'] = pd.to_timedelta(df['Scheduled_time'])
df['Appointment_date'] = df['AppointmentDay'].apply(lambda x: x[:10])
df['Appointment_date'] = pd.to_datetime(df['Appointment_date'])
df['Appointment_time'] = df['AppointmentDay'].apply(lambda x: x[11:-1])
df['Appointment_time'] = pd.to_timedelta(df['Appointment_time'])
df["Appointment_Weekday"] = pd.to_datetime(df.AppointmentDay.apply(lambda x : x[:10])).apply(lambda x: x.weekday())
df["Appointment_month"] = pd.to_datetime(df.AppointmentDay.apply(lambda x : x[:10])).apply(lambda x: x.month)
df["Scheduled_month"] = pd.to_datetime(df.ScheduledDay.apply(lambda x : x[:10])).apply(lambda x: x.month)
df = df.drop(columns = "Appointment_time")

In [5]:
df["differ_day"] = df.Appointment_date - df.Scheduled_date

In [6]:
df.differ_day = df.differ_day.apply(lambda x: int(str(x).split("days")[0]))

### Age가 음수이거나 date_diff가 음수인 데이터 빼주기

In [7]:
df = df[df.Age >= 0]

In [8]:
df = df[df.differ_day >= 0]

### dfx와 dfy로 나눠주기

In [9]:
dfx = df.drop(columns=["PatientId", "AppointmentID", "ScheduledDay", "AppointmentDay", "Age", "No-show", "Scheduled_date", "Scheduled_time", "Appointment_date"], axis=1)
dfy = df["No-show"]

In [10]:
dfx.Handcap = pd.Categorical(dfx.Handcap)
dfx.AppointmentWeekday = pd.Categorical(dfx.Appointment_Weekday)
dfx.Scheduled_month = pd.Categorical(dfx.Scheduled_month)
dfx.Appointment_month = pd.Categorical(dfx.Appointment_month)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [11]:
dfx.tail()

,Gender,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Age_bin,Appointment_Weekday,Appointment_month,Scheduled_month,differ_day
110522,F,MARIA ORTIZ,0,0,0,0,0,1,50s,1,6,5,35
110523,F,MARIA ORTIZ,0,0,0,0,0,1,50s,1,6,5,35
110524,F,MARIA ORTIZ,0,0,0,0,0,1,20s,1,6,4,41
110525,F,MARIA ORTIZ,0,0,0,0,0,1,30s,1,6,4,41
110526,F,MARIA ORTIZ,0,0,0,0,0,1,50s,1,6,4,41


### one hot encoding 적용

In [12]:
dfx = pd.get_dummies(dfx, drop_first=True)

In [13]:
dfx.tail()

,Scholarship,Hipertension,Diabetes,Alcoholism,SMS_received,Appointment_Weekday,differ_day,Gender_M,Neighbourhood_ANDORINHAS,Neighbourhood_ANTÔNIO HONÓRIO,...,Age_bin_90s,Appointment_month_5,Appointment_month_6,Scheduled_month_2,Scheduled_month_3,Scheduled_month_4,Scheduled_month_5,Scheduled_month_6,Scheduled_month_11,Scheduled_month_12
110522,0,0,0,0,1,1,35,0,0,0,...,0,0,1,0,0,0,1,0,0,0
110523,0,0,0,0,1,1,35,0,0,0,...,0,0,1,0,0,0,1,0,0,0
110524,0,0,0,0,1,1,41,0,0,0,...,0,0,1,0,0,1,0,0,0,0
110525,0,0,0,0,1,1,41,0,0,0,...,0,0,1,0,0,1,0,0,0,0
110526,0,0,0,0,1,1,41,0,0,0,...,0,0,1,0,0,1,0,0,0,0


### train과 test 데이터로 나누기

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((77364, 110), (33157, 110), (77364,), (33157,))